In [1]:
## Boiler plate code common to many notebooks.  See the TestFilesCommonCode.ipynb for details
from __future__ import print_function
%run TestFilesCommonCode.ipynb

SimpleITK Version: 0.9.1
Compiled: Sep 28 2015 10:07:41



In [1]:
import os
import glob
import sys

#\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/
#####################################################################################
#     Prepend the shell environment search paths
PROGRAM_PATHS = '/scratch/NAMICExternalProjects/release-20160523/bin'
#PROGRAM_PATHS = '/scratch/BS/release-BSR/bin'
PROGRAM_PATHS = PROGRAM_PATHS.split(':')
PROGRAM_PATHS.extend(os.environ['PATH'].split(':'))
os.environ['PATH'] = ':'.join(PROGRAM_PATHS)

CUSTOM_ENVIRONMENT=dict()

# Platform specific information
#     Prepend the python search paths
#PYTHON_AUX_PATHS = '/scratch/BS/BRAINSTools/AutoWorkup'
PYTHON_AUX_PATHS = '/scratch/SuperResolution/BRAINSSuperResolution/HCPWorkflows/:/scratch/wmql/tract_querier/tract_querier/nipype/'
PYTHON_AUX_PATHS = PYTHON_AUX_PATHS.split(':')
PYTHON_AUX_PATHS.extend(sys.path)
sys.path = PYTHON_AUX_PATHS

import SimpleITK as sitk
import nipype
from nipype.interfaces.base import CommandLine, CommandLineInputSpec, TraitedSpec, File, Directory
from nipype.interfaces.base import traits, isdefined, BaseInterface
from nipype.interfaces.utility import Merge, Split, Function, Rename, IdentityInterface
import nipype.interfaces.io as nio   # Data i/oS
import nipype.pipeline.engine as pe  # pypeline engine
from nipype.interfaces.freesurfer import ReconAll
from nipype.interfaces.ants import DenoiseImage
from nipype.interfaces.semtools import *

In [11]:
input_atlas = '/Shared/johnsonhj/HDNI/ReferenceData/HCP_DATA/test_UKF/100307/wmparc.nii.gz'
input_tractography = '/Shared/johnsonhj/HDNI/ReferenceData/HCP_DATA/test_UKF/100307/2_Rs_0_025/ukfTracts.vtp'

In [4]:
# sr_file_name = os.path.basename(input_tractography)
# print(sr_file_name)
# sr_file_name_base = os.path.splitext(sr_file_name)[0]
# print(sr_file_name_base)
# sr_name = sr_file_name_base.split('_',1)[0]
# print(sr_name)
# out_prefix = sr_name + '_query'
# print(out_prefix)
# output_cst_L = out_prefix + '_' + 'cst.left.vtp'
# output_cst_R = out_prefix + '_' + 'cst.right.vtp'
# print(output_cst_L,output_cst_R)

In [12]:
from wmql import TractQuerier
tract_querier = TractQuerier()
tract_querier.inputs.input_atlas = input_atlas
tract_querier.inputs.input_tractography = input_tractography
tract_querier.inputs.out_prefix = 'query'
tract_querier.inputs.queries = ['cst.left' ,'cst.right']
print(tract_querier.cmdline)
tract_querier.run()

tract_querier -q freesurfer_queries.qry -a /Shared/johnsonhj/HDNI/ReferenceData/HCP_DATA/test_UKF/100307/wmparc.nii.gz -t /Shared/johnsonhj/HDNI/ReferenceData/HCP_DATA/test_UKF/100307/2_Rs_0_025/ukfTracts.vtp -o query --query_selection cst.left,cst.right


In [6]:
input_tractography = '/Volumes/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/HCPWorkflow_CACHE_105115/TractographyWorkflow_CACHE_105115/UKFRunRecordStates/mapflow/_UKFRunRecordStates0/Baseline_ukfTracts.vtp'
input_atlas = '/Shared/sinapse/CACHE/20160610_HCP_base_Results/HCP_DATA/105115/HCP_105115_01/JointFusion/JointFusion_HDAtlas20_2015_fs_standard_label.nii.gz'

In [9]:
def runWMQL(input_tractography, input_atlas):
    import os
    from wmql import TractQuerier
    # prepare proper prefix
    sr_file_name = os.path.basename(input_tractography)
    sr_file_name_base = os.path.splitext(sr_file_name)[0]
    sr_name = sr_file_name_base.split('_',1)[0]
    out_prefix = sr_name + '_query'
    # run WMQL
    tract_querier = TractQuerier()
    tract_querier.inputs.input_atlas = input_atlas
    tract_querier.inputs.input_tractography = input_tractography
    tract_querier.inputs.out_prefix = out_prefix
    tract_querier.inputs.queries = ['cst.left' ,'cst.right']
    print(tract_querier.cmdline)
    tract_querier.run()
    # check outputs
    output_cst_left_name = out_prefix + '_' + 'cst.left.vtp'
    output_cst_right_name = out_prefix + '_' + 'cst.right.vtp'
    output_cst_left = os.path.join(os.getcwd(), output_cst_left_name)
    output_cst_right = os.path.join(os.getcwd(), output_cst_right_name)
    assert os.path.isfile(output_cst_left), "Output cst tract file is not found: %s" % output_cst_left
    assert os.path.isfile(output_cst_right), "Output cst tract file is not found: %s" % output_cst_right
    return [output_cst_left,output_cst_right]

In [10]:
[fn1,fn2] = runWMQL(input_tractography,input_atlas)

tract_querier -q freesurfer_queries.qry -a /Shared/sinapse/CACHE/20160610_HCP_base_Results/HCP_DATA/105115/HCP_105115_01/JointFusion/JointFusion_HDAtlas20_2015_fs_standard_label.nii.gz -t /Volumes/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/HCPWorkflow_CACHE_105115/TractographyWorkflow_CACHE_105115/UKFRunRecordStates/mapflow/_UKFRunRecordStates0/Baseline_ukfTracts.vtp -o Baseline_query --query_selection cst.left,cst.right


AssertionError: Output cst tract file is not found: /Volumes/scratch/Ipython_conda/SimpleITK-Notebook-Answers/Baseline_query_cst.left.vtp